In [6]:
!pip install torch
import torch

zsh:1: /Users/thangpham/Documents/PJ/AutoRace/venv/bin/pip: bad interpreter: /Users/thangpham/Documents/AutoRace/venv/bin/python3.13: no such file or directory


ModuleNotFoundError: No module named 'torch'

In [3]:
import cv2
import numpy as np
import os

def filter_signs_by_color(image):
    hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    # Dải màu đỏ
    lower1, upper1 = np.array([0,70,50]), np.array([10,255,255])
    lower2, upper2 = np.array([170,70,50]), np.array([180,255,255])
    mask_r = cv2.bitwise_or(cv2.inRange(hsv, lower1, upper1), cv2.inRange(hsv, lower2, upper2))
    # Dải màu xanh
    lower3, upper3 = np.array([85,50,200]), np.array([135,250,250])
    mask_b = cv2.inRange(hsv, lower3, upper3)
    # Kết hợp
    return cv2.bitwise_or(mask_r, mask_b)

def get_boxes_from_mask(mask):
    bboxes = []
    numLabels, labels, stats, centroids = cv2.connectedComponentsWithStats(mask, 4, cv2.CV_32S)
    im_height, im_width = mask.shape[:2]
    for i in range(1, numLabels):
        x, y, w, h, area = stats[i]
        if w < 20 or h < 20: continue
        if w > 0.8 * im_width or h > 0.8 * im_height: continue
        if w/h > 2.0 or h/w > 2.0: continue
        bboxes.append((x, y, w, h))
    return bboxes


In [10]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader, random_split

# --- Mô hình CNN (LeNet-like) ---
class TrafficSignCNN(nn.Module):
    def __init__(self, num_classes=7):
        super(TrafficSignCNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, num_classes)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.pool(self.relu(self.conv1(x)))
        x = self.pool(self.relu(self.conv2(x)))
        x = x.view(-1, 16 * 5 * 5)
        x = self.relu(self.fc1(x))
        x = self.relu(self.fc2(x))
        x = self.fc3(x)
        return x

# --- Dataset & DataLoader ---
transform = transforms.Compose([
    transforms.Resize((32, 32)),
    transforms.ToTensor()
])

data_dir = "traffic_sign_images"
dataset = datasets.ImageFolder(root=data_dir, transform=transform)
train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size
train_dataset, test_dataset = random_split(dataset, [train_size, test_size])

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# --- Khởi tạo mô hình ---
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = TrafficSignCNN(num_classes=len(dataset.classes)).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# --- Huấn luyện ---
EPOCHS = 15
for epoch in range(EPOCHS):
    model.train()
    total_loss = 0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    print(f"Epoch [{epoch+1}/{EPOCHS}], Loss: {total_loss/len(train_loader):.4f}")

torch.save(model.state_dict(), "traffic_sign_cnn.pth")
print("✅ Model saved to traffic_sign_cnn.pth")


Epoch [1/15], Loss: 0.0000
Epoch [2/15], Loss: 0.0000
Epoch [3/15], Loss: 0.0000
Epoch [4/15], Loss: 0.0000
Epoch [5/15], Loss: 0.0000
Epoch [6/15], Loss: 0.0000
Epoch [7/15], Loss: 0.0000
Epoch [8/15], Loss: 0.0000
Epoch [9/15], Loss: 0.0000
Epoch [10/15], Loss: 0.0000
Epoch [11/15], Loss: 0.0000
Epoch [12/15], Loss: 0.0000
Epoch [13/15], Loss: 0.0000
Epoch [14/15], Loss: 0.0000
Epoch [15/15], Loss: 0.0000
✅ Model saved to traffic_sign_cnn.pth


In [15]:
import torch
import torch.nn as nn
from torchvision import datasets, transforms

# --- Định nghĩa lại mô hình CNN (LeNet-like) ---
class TrafficSignCNN(nn.Module):
    def __init__(self, num_classes=7):
        super(TrafficSignCNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, num_classes)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.pool(self.relu(self.conv1(x)))
        x = self.pool(self.relu(self.conv2(x)))
        x = x.view(-1, 16 * 5 * 5)
        x = self.relu(self.fc1(x))
        x = self.relu(self.fc2(x))
        x = self.fc3(x)
        return x


transform = transforms.Compose([
    transforms.Resize((32, 32)),
    transforms.ToTensor()
])

dataset = datasets.ImageFolder(root="traffic_sign_images", transform=transform)
classes = dataset.classes

model = TrafficSignCNN(num_classes=len(classes))
model.load_state_dict(torch.load("traffic_sign_cnn.pth", map_location='cpu'))
model.eval()

from PIL import Image

img_path = "traffic_sign_images/stop/1.jpg"  # thử 1 ảnh
image = Image.open(img_path).convert("RGB")
x = transform(image).unsqueeze(0)
pred = model(x)
cls = torch.argmax(pred).item()

print(f"Dự đoán: {classes[cls]}")


Dự đoán: stop


In [19]:
import matplotlib
matplotlib.use("macosx")  # hoặc "TkAgg" nếu bạn dùng cross-platform
import matplotlib.pyplot as plt

import cv2
import numpy as np
import matplotlib.pyplot as plt

def filter_signs_by_color(image):
    """Lọc vùng màu đỏ và xanh (có thể là biển báo)"""
    hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)

    # --- Màu đỏ ---
    lower1, upper1 = np.array([0,70,50]), np.array([10,255,255])
    lower2, upper2 = np.array([170,70,50]), np.array([180,255,255])
    mask1 = cv2.inRange(hsv, lower1, upper1)
    mask2 = cv2.inRange(hsv, lower2, upper2)
    mask_red = cv2.bitwise_or(mask1, mask2)

    # --- Màu xanh ---
    lower3, upper3 = np.array([85,50,200]), np.array([135,250,250])
    mask_blue = cv2.inRange(hsv, lower3, upper3)

    # --- Kết hợp ---
    mask = cv2.bitwise_or(mask_red, mask_blue)
    return mask


ValueError: Key backend: 'module://matplotlib_inline.backend_inline' is not a valid value for backend; supported values are ['gtk3agg', 'gtk3cairo', 'gtk4agg', 'gtk4cairo', 'macosx', 'nbagg', 'notebook', 'qtagg', 'qtcairo', 'qt5agg', 'qt5cairo', 'tkagg', 'tkcairo', 'webagg', 'wx', 'wxagg', 'wxcairo', 'agg', 'cairo', 'pdf', 'pgf', 'ps', 'svg', 'template']